In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io
from glob import glob
from os import listdir

In [3]:
def dfs(img,coordinates,i,j,visited):
    if i<0 or i>=img.shape[0] or j<0 or j>=img.shape[1] or img[i][j]==0 or visited[i][j]==1:
        return
        
    visited[i][j]=1;
    if i<coordinates[0]:
        coordinates[0]=i;
    if i>coordinates[1]:
        coordinates[1]=i;
    if j<coordinates[2]:
        coordinates[2]=j;
    if j>coordinates[3]:
        coordinates[3]=j;
    
    dfs(img,coordinates,i+1,j,visited);
    dfs(img,coordinates,i,j+1,visited);
    dfs(img,coordinates,i-1,j,visited);
    dfs(img,coordinates,i,j-1,visited);

In [4]:
def MarkObject(img):
    
    visited=np.zeros((img.shape[0],img.shape[1]))

    clist=[]
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if img[i][j]==0 or visited[i][j]==1:
                continue
            c=[i,i,j,j]
            dfs(img,c,i,j,visited)
            if c[0]!=c[1] and c[2]!=c[3]:
                clist.append(c)
    return clist

In [5]:
def GetCoordinates(img,istart,iend,jstart,jend):
    
    clist=np.array(MarkObject(img[istart:iend,jstart:jend]))
    ##print(clist)
    if len(clist)>0:
        df=pd.DataFrame({'top':clist[:,0]+istart,'bottom':clist[:,1]+istart,'left':clist[:,2]+jstart,'right':clist[:,3]+jstart})
        #df.to_csv(('coordinates_{}_{}_{}_{}.csv').format(istart,iend,jstart,jend))
        return df
    else:
        return None

In [6]:
folders_composite=glob("Composite Images/*/")
folders_composite

['Composite Images\\20190508-C2GNT1-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT

In [7]:
labels=pd.read_csv('labels.csv',index_col=0)
labels

,label
0,20190508-C2GNT1-EGFP-Noc-01
1,20190508-C2GNT1-EGFP-Noc-02
2,20190508-C2GNT1-EGFP-Noc-03
3,20190508-C2GNT1-EGFP-Noc-04
4,20190508-C2GNT1-EGFP-Noc-05
5,20190508-GALNT4-EGFP-Noc-01
6,20190508-GALNT4-EGFP-Noc-02
7,20190508-GALNT4-EGFP-Noc-03
8,20190508-GALNT4-EGFP-Noc-04
9,20190508-GALNT4-EGFP-Noc-05


In [10]:
for i in range(1,40):

    
    roi_co_file_path=folders_composite[i]+"\\{}-ROI Coordinates.csv".format(labels.iloc[i].values[0])
    renamed_roi_file_path=folders_composite[i]+"\\{}_golgy_coordinates.csv".format(labels.iloc[i].values[0])

    roi_coordinates=pd.read_csv(roi_co_file_path,index_col=0)
    roi_coordinates.to_csv(renamed_roi_file_path)

    print('Done with {}'.format(labels.iloc[i].values[0]))

Done with 20190508-C2GNT1-EGFP-Noc-02
Done with 20190508-C2GNT1-EGFP-Noc-03
Done with 20190508-C2GNT1-EGFP-Noc-04
Done with 20190508-C2GNT1-EGFP-Noc-05
Done with 20190508-GALNT4-EGFP-Noc-01
Done with 20190508-GALNT4-EGFP-Noc-02
Done with 20190508-GALNT4-EGFP-Noc-03
Done with 20190508-GALNT4-EGFP-Noc-04
Done with 20190508-GALNT4-EGFP-Noc-05
Done with 20190508-GALNT8-EGFP-Noc-01
Done with 20190508-GALNT8-EGFP-Noc-02
Done with 20190508-GALNT8-EGFP-Noc-03
Done with 20190508-GALNT8-EGFP-Noc-04
Done with 20190508-GALNT8-EGFP-Noc-05
Done with 20190508-GALNT8-EGFP-Noc-06
Done with 20190508-GALNT8-EGFP-Noc-07
Done with 20190508-GALNT8-EGFP-Noc-08
Done with 20190508-tetraspeckbeads-aft-01
Done with 20190508-tetraspeckbeads-bef-01
Done with 20190508-VIP36-EGFP-Noc-01
Done with 20190508-VIP36-EGFP-Noc-02
Done with 20190508-VIP36-EGFP-Noc-03
Done with 20190508-VIP36-EGFP-Noc-04
Done with 20190508-VIP36-EGFP-Noc-05
Done with 20190508-VIP36-EGFP-Noc-06
Done with 20190508-VIP36-EGFP-Noc-07
Done with 2

In [11]:
sys.setrecursionlimit(1000000)

In [29]:
for k in range(1,40):
    img_obj_file_path=folders_composite[k]+"all_object.tif"
    img_obj=io.imread(img_obj_file_path)
    
    img_non_golgy_file_path=folders_composite[k]+"non_golgy_object.tif"
    img_non_golgy_obj=io.imread(img_non_golgy_file_path)
    
    roi_co_file_path=folders_composite[k]+"\\{}-ROI Coordinates.csv".format(labels.iloc[k].values[0])
    old_roi_co_file_path=folders_composite[k]+"\\{}-ROI Coordinates - old.csv".format(labels.iloc[k].values[0])

    non_golgi_df=pd.DataFrame(columns=['top','bottom','left','right'])
    obj_df=pd.DataFrame(columns=['top','bottom','left','right'])

    ## Extract coordinates for all objects
    i=0
    j=0
    while i<=1080:
        iend=(i+50) if i+50<1080 else 1080
        while j<=1280:
            jend=(j+50) if j+50<1280 else 1280
            ##print('Now {},{}'.format(i,j))
            coordinates=GetCoordinates(img_obj,i,i+50,j,j+50)
            if coordinates is not None:
                obj_df=obj_df.append(coordinates).reset_index(drop=True)
            j+=50
        i+=50
        j=0 
    
    ## Extract coordinates for non-golgy objects
    i=0
    j=0
    while i<=1080:
        iend=(i+50) if i+50<1080 else 1080
        while j<=1280:
            jend=(j+50) if j+50<1280 else 1280
            ##print('Now {},{}'.format(i,j))
            coordinates=GetCoordinates(img_non_golgy_obj,i,i+50,j,j+50)
            if coordinates is not None:
                non_golgi_df=non_golgi_df.append(coordinates).reset_index(drop=True)
            j+=50
        i+=50
        j=0 


    obj_co_file_path=folders_composite[k]+"\\{}_all_coordinates.csv".format(labels.iloc[k].values[0])
    non_golgy_obj_co_file_path=folders_composite[k]+"\\{}_non_golgy_coordinates.csv".format(labels.iloc[k].values[0])

    obj_df.to_csv(obj_co_file_path)
    non_golgi_df.to_csv(non_golgy_obj_co_file_path)

    print('Done with {}'.format(labels.iloc[k].values[0]))

Done with 20190508-C2GNT1-EGFP-Noc-02
Done with 20190508-C2GNT1-EGFP-Noc-03
Done with 20190508-C2GNT1-EGFP-Noc-04
Done with 20190508-C2GNT1-EGFP-Noc-05
Done with 20190508-GALNT4-EGFP-Noc-01
Done with 20190508-GALNT4-EGFP-Noc-02
Done with 20190508-GALNT4-EGFP-Noc-03
Done with 20190508-GALNT4-EGFP-Noc-04
Done with 20190508-GALNT4-EGFP-Noc-05
Done with 20190508-GALNT8-EGFP-Noc-01
Done with 20190508-GALNT8-EGFP-Noc-02
Done with 20190508-GALNT8-EGFP-Noc-03
Done with 20190508-GALNT8-EGFP-Noc-04
Done with 20190508-GALNT8-EGFP-Noc-05
Done with 20190508-GALNT8-EGFP-Noc-06
Done with 20190508-GALNT8-EGFP-Noc-07
Done with 20190508-GALNT8-EGFP-Noc-08
Done with 20190508-tetraspeckbeads-aft-01
Done with 20190508-tetraspeckbeads-bef-01
Done with 20190508-VIP36-EGFP-Noc-01
Done with 20190508-VIP36-EGFP-Noc-02
Done with 20190508-VIP36-EGFP-Noc-03
Done with 20190508-VIP36-EGFP-Noc-04
Done with 20190508-VIP36-EGFP-Noc-05
Done with 20190508-VIP36-EGFP-Noc-06
Done with 20190508-VIP36-EGFP-Noc-07
Done with 2